In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.nn.functional as F


import torchvision
from torchvision.models import resnet18, ResNet18_Weights, vgg16, VGG16_Weights
from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


import os
import sys
import copy
from torchsummary import summary


# from google.colab import drive
# drive.mount('/content/drive')
# #Code folder path
# %cd /content/drive/My Drive/ECE591_DL_CL_PROJECT/

#device config
torch.cuda.set_device(0)#Select GPU device 1
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Get Packages
sys.path.append('./Tools')

#Network architectures
from ResNet18_CIFAR10 import ResNet18_CIFAR10_Model
from ResNet18_TINYIMAGENET import ResNet18_TINYIMAGENET_Model
from VGG16_CIFAR10 import VGG16_CIFAR10_Model
from VGG16_TINYIMAGENET import VGG16_TINYIMAGENET_Model

#Non Corrupted DataLoaders
from CIFAR10_LOADER import CIFAR10DataLoader
from TINYIMAGENET_LOADER import TINYIMAGENETDataLoader

#Non Corrupted DataLoaders
from CIFAR10C_LOADER import CIFAR10C_DataLoader #Each Corruption is a entry in the dictionary
from TINYIMAGENET_C_LOADER import TinyImagenetC_DataLoader #Each Corruption is a entry in the dictionary

#Error Metrics
from ErrorMetrics import Evaluate_Model_TOP1
from GeneralPruning import iterative_pruning


# Unpruned Models

In [2]:
#Network weights
ResNet18_CIFAR10_Weights= torch.load("./Networks/ResNet18_CIFAR10_V0.pt")
ResNet18_TINYIMAGENET_Weights=torch.load("./Networks/ResNet18_TINY_IMAGENET_V0.pt")
VGG16_CIFAR10_Weights=torch.load("./Networks/VGG16_CIFAR10_V0.pt")
VGG16_TINYIMAGENET_Weights=torch.load("./Networks/VGG16_TINY_IMAGENET_V0.pt")

#Unpruned Models
ResNet18_CIFAR10_0P=ResNet18_CIFAR10_Model()
ResNet18_CIFAR10_0P.load_state_dict(ResNet18_CIFAR10_Weights)

ResNet18_TINYIMAGENET_0P=ResNet18_TINYIMAGENET_Model()
ResNet18_TINYIMAGENET_0P.load_state_dict(ResNet18_TINYIMAGENET_Weights)

VGG16_CIFAR10_0P=VGG16_CIFAR10_Model()
VGG16_CIFAR10_0P.load_state_dict(VGG16_CIFAR10_Weights)

VGG16_TINYIMAGENET_0P=VGG16_TINYIMAGENET_Model()
VGG16_TINYIMAGENET_0P.load_state_dict(VGG16_TINYIMAGENET_Weights)


<All keys matched successfully>

# Load Datasets

In [3]:
from unittest import loader

Times=5
trainloader_CIFAR10, testloader_CIFAR10=CIFAR10DataLoader(batch_size=512*Times) #Load Data for CIFAR10
loader_CIFAR10C=CIFAR10C_DataLoader(batch_size=512*Times) #Load Data for CIFAR10C

Times=1
trainloader_TINYIMAGENET, testloader_TINYIMAGENET=TINYIMAGENETDataLoader(NPZ=True,batch_size=512*Times) #Load Data for TINYIMAGENET
loader_TinyImagenetC=TinyImagenetC_DataLoader(NPZ=True,batch_size=512*Times) #Load Data for TINYIMAGENETC

Files already downloaded and verified
Files already downloaded and verified


# CIFAR 10

## Check Effect of Pruning Rate

In [5]:
#RestNet18
model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.5
NumEpochsEachPruneRetrain=50
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR_05PR_50Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 50 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.1358, Accuracy: 47844/50000 (95.69%)
Test set: Average loss: 0.3436, Accuracy: 8904/10000 (89.04%)

--Testing Corruptions--


100%|██████████| 19/19 [03:44<00:00, 11.82s/it]



Avg Corruption Accuracy: 76.10%
---------- Pruning Iteration:  1 / 7


Loss=0.15581966936588287 Batch_id=19 Accuracy=94.77: 100%|██████████| 20/20 [00:38<00:00,  1.93s/it]
Loss=0.16265031695365906 Batch_id=19 Accuracy=94.77: 100%|██████████| 20/20 [00:36<00:00,  1.83s/it]
Loss=0.1636085957288742 Batch_id=19 Accuracy=94.91: 100%|██████████| 20/20 [00:39<00:00,  1.96s/it] 
Loss=0.14132516086101532 Batch_id=19 Accuracy=95.15: 100%|██████████| 20/20 [00:39<00:00,  1.96s/it]
Loss=0.13396283984184265 Batch_id=19 Accuracy=95.40: 100%|██████████| 20/20 [00:36<00:00,  1.81s/it]
Loss=0.14013925194740295 Batch_id=19 Accuracy=95.62: 100%|██████████| 20/20 [00:39<00:00,  1.96s/it]
Loss=0.13257646560668945 Batch_id=19 Accuracy=95.73: 100%|██████████| 20/20 [00:38<00:00,  1.95s/it]
Loss=0.11023104190826416 Batch_id=19 Accuracy=95.63: 100%|██████████| 20/20 [00:38<00:00,  1.90s/it]
Loss=0.1282247006893158 Batch_id=19 Accuracy=95.87: 100%|██████████| 20/20 [00:38<00:00,  1.93s/it] 
Loss=0.11265090852975845 Batch_id=19 Accuracy=96.11: 100%|██████████| 20/20 [00:38<00:00,  

Epoch 00021: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00030: reducing learning rate of group 0 to 3.1250e-09.


Loss=0.5469545722007751 Batch_id=19 Accuracy=81.14: 100%|██████████| 20/20 [00:39<00:00,  1.99s/it]
Loss=0.54313063621521 Batch_id=19 Accuracy=81.53: 100%|██████████| 20/20 [00:37<00:00,  1.89s/it]  
Loss=0.554857075214386 Batch_id=19 Accuracy=81.21: 100%|██████████| 20/20 [00:40<00:00,  2.01s/it] 
Loss=0.5038308501243591 Batch_id=19 Accuracy=81.20: 100%|██████████| 20/20 [00:39<00:00,  1.99s/it]
Loss=0.5754631757736206 Batch_id=19 Accuracy=81.22: 100%|██████████| 20/20 [00:39<00:00,  1.96s/it]
Loss=0.535887598991394 Batch_id=19 Accuracy=81.24: 100%|██████████| 20/20 [00:36<00:00,  1.84s/it] 
Loss=0.5554140210151672 Batch_id=19 Accuracy=81.59: 100%|██████████| 20/20 [00:39<00:00,  1.97s/it]
Loss=0.5268827676773071 Batch_id=19 Accuracy=81.27: 100%|██████████| 20/20 [00:39<00:00,  1.96s/it]
Loss=0.5674397945404053 Batch_id=19 Accuracy=81.02: 100%|██████████| 20/20 [00:37<00:00,  1.87s/it]
Loss=0.5551440715789795 Batch_id=19 Accuracy=81.28: 100%|██████████| 20/20 [00:39<00:00,  1.98s/it]


Test set: Average loss: 0.5126, Accuracy: 8223/10000 (82.23%)

--Testing Corruptions--


100%|██████████| 19/19 [04:07<00:00, 13.03s/it]


Avg Corruption Accuracy: 65.05%
Global Sparsity: 99.17% 

